In [1]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data_dir = 'C:/Users/mauli/Desktop/SOIL CLASSIFICATION DATA/MAIN/train'

In [3]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2,
    horizontal_flip = False,
    zoom_range = 0.2,
    shear_range = 0.2,
)

train_generator =datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),
    batch_size =32,
    class_mode = 'categorical',
    subset = 'training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),
    batch_size =32,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 6146 images belonging to 8 classes.
Found 1532 images belonging to 8 classes.


In [4]:
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom layers on top of the base model
model = models.Sequential([
    base_model,  

    layers.Flatten(),
    
    layers.Dense(512, activation='relu'),
    # Output layer for 5 classes with softmax activation
    layers.Dense(8, activation='softmax')
])

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
history = model.fit(
    train_generator,
    epochs=1,  # You can adjust the number of epochs
    validation_data=validation_generator
)

C:\Users\mauli\Desktop\python_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


193/193 ━━━━━━━━━━━━━━━━━━━━ 720s 3s/step - accuracy: 0.6327 - loss: 5.8400 - val_accuracy: 0.7696 - val_loss: 0.8591


In [8]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')

48/48 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.7696 - loss: 0.8756
Validation Accuracy: 0.77
